## Overview

There are three primary conditions and all trials have the following structure:

<img src="../support/trial_design.png" alt="trial" style="width: 500px;"/>

Three conditions:
    
    precue, postcue, nocue
    10% of trials have both precue and postcue - 
    
        object, scene
        scene, object
        scene, scene
        object object



RSVP:
    8 categories
    6 images - three from scene/ three from object whether target is present or not, both are balanced
        random sample from 4 scenes, and from 4 objects
    
    


In [17]:
import itertools
import pandas as pd
import numpy as np
import random
from copy import deepcopy

In [18]:
subject = 's028'

In [19]:
random.seed(subject) #s001, practice

In [20]:
## Three Steps: 
## 1) Generate all your counterbalancing and trials and put them into a dataframe 
## 2) Define experimental structure
## 3) Fit the trials into experimental structure (e.g. assigning targets to cue part, assigning stimuli 
#to single-stimulus parts etc. This is the most complicated part.

In [21]:
def flatten_all_tuples(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten_all_tuples(i):
                yield j
        else:
            yield i

### Constructing the RSVP orders

There are 6 images in the RSVP stream. To begin with, there are 6 places where the target (`1`) can appear among the distractors (`'d'`)

In [22]:
innums = [1,'d','d','d','d','d']
keep = [item for item in itertools.permutations(innums,6)]
keep = list(set(keep))
keep.sort(reverse=False)
keep

[(1, 'd', 'd', 'd', 'd', 'd'),
 ('d', 1, 'd', 'd', 'd', 'd'),
 ('d', 'd', 1, 'd', 'd', 'd'),
 ('d', 'd', 'd', 1, 'd', 'd'),
 ('d', 'd', 'd', 'd', 1, 'd'),
 ('d', 'd', 'd', 'd', 'd', 1)]

There are eight target identities, so I generate a list where each target identity appears at each of the 6 target locations:

In [23]:
# Eight Target Identities
scenes = ['woods','bathroom','desert','coast']
objects = ['flower','car','shoe','chair'] 
stimuli = [scenes, objects]
target_identities = ['woods','bathroom','desert','coast','flower','car','shoe','chair']
trials = list(itertools.product(target_identities,keep))

Then I take that list (of target identity x target location) and use it for each of the three trial types:

In [24]:
#Three cue types
trial_types = ['nocue','precue','postcue']
trials = list(itertools.product(trial_types,trials))

And repeat it again for target type (absent vs present):

In [25]:
#Two target present types
target_types = ['absent','present']
trials = list(itertools.product(target_types,trials))

And finally, repeat the list again for how many times I want the same type of trial to repeat (3 times):

In [26]:
#How many times is the whole sequence repeated
repeat_num = [1,2,3]
trials = list(itertools.product(repeat_num,trials))

As a result, I should have a dataframe that has 6x8x3x2x3 (=864) trials:

In [32]:
all_trials = [list(flatten_all_tuples(item)) for item in trials]
df = pd.DataFrame(all_trials)
df.columns = ['repeat_num','target_presence','cue_type','target_identity','first','second','third','fourth','fifth','sixth']
print df.shape

(864, 10)


Of course, this is far more trials than I can actually use. There are several key constraints that need to be imposed to make the trials valid.

First, I only want to keep trials where the target is NOT the first or sixth stimulus:

In [34]:
df = df[df['first'] != 1]
df = df[df['sixth'] != 1]

Second, for trials in the `'nocue'` condition, the target is undefined, so there is no target, no target location (in the RSVP stream), and no target category:

In [37]:
df.loc[(df.cue_type == 'nocue'), 'target_identity'] = False
df.loc[(df.cue_type == 'nocue'), 'target_location'] = False
df.loc[(df.cue_type == 'nocue'), 'target_category'] = False

Third, for trials in the `'nocue'` condition where the target is undefined, all the images are effectictively distractors and should be labeled as such:

In [38]:
df.loc[df['cue_type'] == 'nocue', 'first'] = 'd'
df.loc[df['cue_type'] == 'nocue', 'second'] = 'd'
df.loc[df['cue_type'] == 'nocue', 'third'] = 'd'
df.loc[df['cue_type'] == 'nocue', 'fourth'] = 'd'
df.loc[df['cue_type'] == 'nocue', 'fifth'] = 'd'
df.loc[df['cue_type'] == 'nocue', 'sixth'] = 'd'

In [39]:
df.loc[df['target_presence'] == 'absent', 'first'] = 'd'
df.loc[df['target_presence'] == 'absent', 'second'] = 'd'
df.loc[df['target_presence'] == 'absent', 'third'] = 'd'
df.loc[df['target_presence'] == 'absent', 'fourth'] = 'd'
df.loc[df['target_presence'] == 'absent', 'fifth'] = 'd'
df.loc[df['target_presence'] == 'absent', 'sixth'] = 'd'

In [275]:
df.loc[df.target_identity.isin(stimuli[0]), 'target_category'] = 'scene'
df.loc[df.target_identity.isin(stimuli[1]), 'target_category'] = 'object'

In [276]:
df.loc[df['first'] == 1, 'first'] = df.target_identity
df.loc[df['second'] == 1, 'second'] = df.target_identity
df.loc[df['third'] == 1, 'third'] = df.target_identity
df.loc[df['fourth'] == 1, 'fourth'] = df.target_identity
df.loc[df['fifth'] == 1, 'fifth'] = df.target_identity
df.loc[df['sixth'] == 1, 'sixth'] = df.target_identity

In [279]:
df_new = df[['target_identity','first','second','third','fourth','fifth','sixth']]

In [280]:
matches = df_new[df_new.columns.difference(['target_identity'])].eq(df_new['target_identity'], axis = 0)
df_new["target_location"] = matches.apply(lambda x: ", ".join(x.index[np.where(x.tolist())]), axis = 1)

/Users/Amyunimus/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [281]:
df_new["exists"] = df_new.drop("target_identity", 1).eq(df_new["target_identity"], 0).any(1)

/Users/Amyunimus/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [282]:
df["exists"] = df_new["exists"]
df["target_location"] = df_new["target_location"]

In [283]:
df.loc[df["target_location"] == "", "target_location"] = False

In [284]:
df.head()

,repeat_num,target_presence,cue_type,target_identity,first,second,third,fourth,fifth,sixth,target_location,target_category,exists
1,1,absent,nocue,False,d,d,d,d,d,d,False,False,False
2,1,absent,nocue,False,d,d,d,d,d,d,False,False,False
3,1,absent,nocue,False,d,d,d,d,d,d,False,False,False
4,1,absent,nocue,False,d,d,d,d,d,d,False,False,False
7,1,absent,nocue,False,d,d,d,d,d,d,False,False,False


In [285]:
#get rid of trails where there is "target present" and "no cue" since that isn't possible
#This also reduces the number of trials in no_cue by half so that NOT getting a cue isn't predictive of target absence 
df = df[~((df.cue_type == 'nocue') & (df.target_presence == 'present'))]
df.shape

(480, 13)

In [286]:
pd.crosstab([df.target_identity,df.target_presence], df.cue_type, margins=True)

cue_type                         nocue  postcue  precue  All
target_identity target_presence                             
False           absent              96        0       0   96
bathroom        absent               0       12      12   24
                present              0       12      12   24
car             absent               0       12      12   24
                present              0       12      12   24
chair           absent               0       12      12   24
                present              0       12      12   24
coast           absent               0       12      12   24
                present              0       12      12   24
desert          absent               0       12      12   24
                present              0       12      12   24
flower          absent               0       12      12   24
                present              0       12      12   24
shoe            absent               0       12      12   24
                present              0       12      12   24
woods           absent               0       12      12   24
                present              0       12      12   24
All                                 96      192     192  480

In [287]:
#Get rid of all trials where the target is absent, but listed as one of the stimulis 
df = df[~((df.target_presence == 'absent') & (df["exists"]== True))]

In [288]:
pd.crosstab([df['exists'],df.target_presence], df.cue_type, margins=True)

,cue_type,nocue,postcue,precue,All
exists,target_presence,,,,
False,absent,96,96,96,288
True,present,0,96,96,192
All,,96,192,192,480


In [289]:
#Get rid of all trials where the target is present, but not listed as one of the stimulis 
df = df[~((df.target_presence == 'present') & (df["exists"]== False))]

df.shape

(480, 13)

In [290]:
pd.crosstab([df['exists'],df.target_presence], df.cue_type, margins=True)

,cue_type,nocue,postcue,precue,All
exists,target_presence,,,,
False,absent,96,96,96,288
True,present,0,96,96,192
All,,96,192,192,480


In [291]:
#all positions are even for all categories (but there will be some rsvp_orders that are seen twice as much -- second and fourth positions, third and fifth positions)
pd.crosstab([df['exists'],df.target_presence], df.cue_type, margins=True)

,cue_type,nocue,postcue,precue,All
exists,target_presence,,,,
False,absent,96,96,96,288
True,present,0,96,96,192
All,,96,192,192,480


In [292]:
pd.crosstab(df.target_location, df.cue_type, margins=True)

cue_type,nocue,postcue,precue,All
target_location,,,,
False,96,96,96,288
fifth,0,24,24,48
fourth,0,24,24,48
second,0,24,24,48
third,0,24,24,48
All,96,192,192,480


In [293]:
pd.crosstab(df.target_location, df.target_identity, margins=True)

target_identity,False,bathroom,car,chair,coast,desert,flower,shoe,woods,All
target_location,,,,,,,,,,
False,96,24,24,24,24,24,24,24,24,288
fifth,0,6,6,6,6,6,6,6,6,48
fourth,0,6,6,6,6,6,6,6,6,48
second,0,6,6,6,6,6,6,6,6,48
third,0,6,6,6,6,6,6,6,6,48
All,96,48,48,48,48,48,48,48,48,480


In [294]:
df = df.reset_index(drop=True)

In [295]:
#73.3*2.2 - 144 

In [296]:
loc_dict = {'second':1,'third':2,'fourth':3,'fifth':4}
cat_dict = {'object':objects, 'scene':scenes}
for i,row in df[df.cue_type == 'precue'].iterrows():
    distractors1 = random.sample(objects,3)   
    distractors2 = random.sample(scenes,3) 
    distractors = distractors1 + distractors2
    random.shuffle(distractors)
    
    #while not tmp:
    #    tmp = distractors[loc_dict[row['target_location']]] in cat_dict[row['target_category']]

    equal_scenes_objects = False
    while not equal_scenes_objects:
        while row['target_identity'] in distractors:            

                distractors1 = random.sample(objects,3)   
                distractors2 = random.sample(scenes,3) 
                distractors = distractors1 + distractors2
                random.shuffle(distractors)
                        
        try:
            distractors[loc_dict[row['target_location']]] = row['target_identity']
            equal_scenes_objects = len(set.intersection(set(distractors), set(scenes))) == len(set.intersection(set(distractors), set(objects)))
        
        except KeyError:
            equal_scenes_objects = len(set.intersection(set(distractors), set(scenes))) == len(set.intersection(set(distractors), set(objects)))



    if not len(set.intersection(set(distractors), set(scenes))) == len(set.intersection(set(distractors), set(objects))):
        print distractors
    #print len(set.intersection(set(distractors), set(scenes))), len(set.intersection(set(distractors), set(objects)))
    for j,item in enumerate(['first','second','third','fourth','fifth','sixth']):
        if df.loc[i, item] == 'd':
            df.loc[i, item] = distractors[j]
            
            
        

In [297]:
#Check if equal (they should be)
c = df[df.cue_type == 'precue'][['target_identity','target_location']].reset_index() == df[df.cue_type == 'postcue'][['target_identity','target_location']].reset_index()
print c.target_identity.unique()
print c.target_location.unique()


[True]
[True]


In [298]:
##Assign the orders we just made for precue to nocue
for item in ['first','second','third','fourth','fifth','sixth']:
    df.loc[df.cue_type == 'postcue', item] = df[df.cue_type == 'precue'][item].values

In [299]:
##Assign the orders we just made for precue to nocue
for item in ['first','second','third','fourth','fifth','sixth']:
    df.loc[df.cue_type == 'nocue', item] = df[(df.cue_type == 'precue') & (df.target_presence == 'present')][item].values

In [300]:
df[(df.cue_type == 'precue') & (df.target_presence == 'present')]['first'].shape

(96,)

In [301]:
df['rsvp_order'] = df['first'].astype(str) +","+ df['second'].astype(str) +","+ \
df['third'].astype(str)+","+ df['fourth'].astype(str)+","+ df['fifth'].astype(str) +","+ df['sixth'].astype(str)

In [302]:
df.tail()

,repeat_num,target_presence,cue_type,target_identity,first,second,third,fourth,fifth,sixth,target_location,target_category,exists,rsvp_order
475,3,present,postcue,shoe,bathroom,woods,coast,car,shoe,flower,fifth,object,True,"bathroom,woods,coast,car,shoe,flower"
476,3,present,postcue,chair,shoe,chair,car,coast,woods,bathroom,second,object,True,"shoe,chair,car,coast,woods,bathroom"
477,3,present,postcue,chair,desert,shoe,chair,woods,coast,car,third,object,True,"desert,shoe,chair,woods,coast,car"
478,3,present,postcue,chair,coast,shoe,flower,chair,bathroom,desert,fourth,object,True,"coast,shoe,flower,chair,bathroom,desert"
479,3,present,postcue,chair,flower,bathroom,shoe,coast,chair,desert,fifth,object,True,"flower,bathroom,shoe,coast,chair,desert"


In [303]:
df_tmp = deepcopy(df)

In [304]:
df_tmp[['first','second','third','fourth','fifth','sixth']]= df_tmp[['first','second','third','fourth','fifth','sixth']].replace(
    {'woods':0,'bathroom':0,'desert':0,'coast':0,
     'flower':1,'car':1,'shoe':1,'chair':1})

In [305]:
df_tmp['num_scenes'] = df_tmp['first'] + df_tmp['second'] + df_tmp['third']+ df_tmp['fourth']+ df_tmp['fifth']+ df_tmp['sixth']

#df_tmp[['first','second','third','fourth','fifth','sixth']].stack().value_counts()

In [306]:
df_tmp[(df_tmp.cue_type == 'postcue') & (df_tmp.target_presence == 'present')]['num_scenes'].unique()

array([3])

In [307]:
df_tmp[['first','second','third','fourth','fifth','sixth']].stack().value_counts()

1    1440
0    1440
dtype: int64

In [308]:
df.shape

(480, 14)

In [309]:
#df.iloc[np.random.permutation(len(df))].reset_index(drop=True)

In [310]:
final_df = df

In [311]:
#randomizes trials by repeat (3 here), so that every trial is scene once before repeating
final_df = df.groupby('repeat_num').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)



In [312]:
final_df.head()

,repeat_num,target_presence,cue_type,target_identity,first,second,third,fourth,fifth,sixth,target_location,target_category,exists,rsvp_order
0,1,absent,nocue,False,bathroom,woods,coast,shoe,car,flower,False,False,False,"bathroom,woods,coast,shoe,car,flower"
1,1,present,postcue,desert,bathroom,woods,desert,shoe,car,chair,third,scene,True,"bathroom,woods,desert,shoe,car,chair"
2,1,present,precue,desert,bathroom,woods,desert,shoe,car,chair,third,scene,True,"bathroom,woods,desert,shoe,car,chair"
3,1,present,postcue,flower,desert,flower,coast,bathroom,car,chair,second,object,True,"desert,flower,coast,bathroom,car,chair"
4,1,present,precue,desert,woods,desert,coast,shoe,chair,flower,second,scene,True,"woods,desert,coast,shoe,chair,flower"


In [313]:
df.groupby("second").cumcount()+1

0       1
1       1
2       2
3       1
4       1
5       1
6       2
7       1
8       1
9       2
10      3
11      2
12      2
13      3
14      3
15      1
16      3
17      4
18      3
19      2
20      5
21      4
22      4
23      5
24      2
25      4
26      5
27      2
28      4
29      5
       ..
450    63
451    54
452    51
453    64
454    65
455    48
456    49
457    70
458    61
459    66
460    62
461    50
462    67
463    55
464    68
465    41
466    63
467    51
468    56
469    52
470    53
471    70
472    71
473    52
474    64
475    71
476    42
477    72
478    73
479    54
dtype: int64

In [314]:
final_df.groupby(['repeat_num','cue_type'])['target_category'].count()

repeat_num  cue_type
1           nocue       32
            postcue     64
            precue      64
2           nocue       32
            postcue     64
            precue      64
3           nocue       32
            postcue     64
            precue      64
Name: target_category, dtype: int64

In [315]:
##Add catch trials

In [316]:
# doublecue
# Precue target can be present or absent
# postcue target can be present or absent

innums = [1,1,'d','d','d','d']
keep = [item for item in itertools.permutations(innums,6)]
keep = list(set(keep))
keep.sort(reverse=False)


target_types = ['absent','present']
absent_present_combo = [item for item in itertools.product(target_types,repeat=2)]

target_categories = ['scene','object']
target_category_combo = [item for item in itertools.permutations(target_categories)]

target_order = ['scene','object']
target_order_combo = [item for item in itertools.permutations(target_order)]





scenes = ['woods','bathroom','desert','coast']
objects = ['flower','car','shoe','chair'] 
target_identities = ['woods','bathroom','desert','coast','flower','car','shoe','chair']
trials = list(itertools.product(scenes,objects))
stimuli_combo = trials




trials = list(itertools.product(absent_present_combo,target_category_combo))
trials = list(itertools.product(trials,target_order_combo))


# #keep= all_rsvp_orders
# # Four Target Identities
# scenes = ['woods','bathroom','desert','coast']
# objects = ['flower','car','shoe','chair'] 
# target_identities = ['woods','bathroom','desert','coast','flower','car','shoe','chair']
# trials = list(itertools.product(target_identities,keep))

# #Three cue types
# trial_types = ['doublecue']
# trials = list(itertools.product(trial_types,trials))

# target_types = ['absent','present']
# trials = list(itertools.product(target_types,trials))

#How many times is the whole sequence repeated
repeat_num = [1,2,3]
trials = list(itertools.product(repeat_num,trials))



In [317]:
trials.sort()
len(trials)

48

In [318]:
all_trials = [list(flatten_all_tuples(item)) for item in trials]

In [319]:
df = pd.DataFrame(all_trials)
print df.shape

(48, 7)


In [320]:
stimuli_combo.sort()

In [321]:
stimuli_combo

[('bathroom', 'car'),
 ('bathroom', 'chair'),
 ('bathroom', 'flower'),
 ('bathroom', 'shoe'),
 ('coast', 'car'),
 ('coast', 'chair'),
 ('coast', 'flower'),
 ('coast', 'shoe'),
 ('desert', 'car'),
 ('desert', 'chair'),
 ('desert', 'flower'),
 ('desert', 'shoe'),
 ('woods', 'car'),
 ('woods', 'chair'),
 ('woods', 'flower'),
 ('woods', 'shoe')]

In [322]:
stimuli_combo1 = deepcopy(stimuli_combo)
stimuli_combo2 = deepcopy(stimuli_combo)
stimuli_combo3 = deepcopy(stimuli_combo)
random.shuffle(stimuli_combo1)
random.shuffle(stimuli_combo2)
random.shuffle(stimuli_combo3)

In [323]:
stim_pairs = stimuli_combo1 + stimuli_combo2+ stimuli_combo3

In [324]:
#These are good for each rep (16)
#Stimulus needs to be assigned (and will be random?) Each type will be present 
#Location needs to be assigned (and will be random)
df.columns = ['repeat_num','target_presence1','target_presence2','target_category1','target_category2','first_target','second_target']

In [325]:
df['scene'] = [item[0] for item in stim_pairs]
df['object'] = [item[1] for item in stim_pairs]

In [326]:
df.loc[df.target_presence1 == 'absent','target_category1'] = False
df.loc[df.target_presence1 == 'absent','first_target'] = False
df.loc[df.target_presence2 == 'absent','target_category2'] = False
df.loc[df.target_presence2 == 'absent','second_target'] = False

In [327]:
df['target_presence'] = zip(df.target_presence1,df.target_presence2)
df['target_category'] = zip(df.first_target, df.second_target)

In [328]:
df.head()

,repeat_num,target_presence1,target_presence2,target_category1,target_category2,first_target,second_target,scene,object,target_presence,target_category
0,1,absent,absent,False,False,False,False,woods,flower,"(absent, absent)","(False, False)"
1,1,absent,absent,False,False,False,False,woods,shoe,"(absent, absent)","(False, False)"
2,1,absent,absent,False,False,False,False,bathroom,shoe,"(absent, absent)","(False, False)"
3,1,absent,absent,False,False,False,False,woods,chair,"(absent, absent)","(False, False)"
4,1,absent,present,False,scene,False,scene,woods,car,"(absent, present)","(False, scene)"


In [329]:
target_list = []
for i,row in df.iterrows():
    if row.first_target == 'object':
        item1 = df.loc[i, 'object']
    elif row.first_target == 'scene':
        item1 = df.loc[i, 'scene']
    elif row.first_target == False:
        item1 = False
    else:
        print "NA"
    
    if row.second_target == 'object':
        item2 = df.loc[i, 'object']
    elif row.second_target == 'scene':
        item2 = df.loc[i, 'scene']
    elif row.second_target == False:
        item2 = False
    else:
        print "NA"
    
        
        
    target_list.append((item1, item2))


    

In [330]:
df['target_identity'] = target_list

In [331]:
innums = [1,1,'d','d','d','d']
keep = [item for item in itertools.permutations(innums,6)]
keep = list(set(keep))
keep.sort(reverse=False)

In [332]:
double_cues_rsvps = [item for item in keep if item[0] != 1 and item[-1] !=1]

In [333]:
print len(double_cues_rsvps)

6


In [334]:
invals = pd.Series(double_cues_rsvps*2).values#[:8]

In [335]:
random.shuffle(invals)

In [336]:
df[df.target_presence == ('present', 'present')].shape

(12, 12)

In [337]:
df['rsvp_order'] = 0
df.loc[df.target_presence == ('present', 'present'), 'rsvp_order'] = invals


In [338]:
innums = [1,'d','d','d','d','d']
keep = [item for item in itertools.permutations(innums,6)]
keep = list(set(keep))
keep.sort(reverse=False)
single_cues_rsvps = [item for item in keep if item[0] != 1 and item[-1] !=1]

In [339]:
print len(single_cues_rsvps)

4


In [340]:
invals = pd.Series(single_cues_rsvps*6).values

In [341]:
random.shuffle(invals)

In [342]:
df[(df.target_presence == ('absent', 'present')) | (df.target_presence == ('present', 'absent')) ].shape

(24, 13)

In [343]:
df.loc[(df.target_presence == ('absent', 'present')) | (df.target_presence == ('present', 'absent')), 'rsvp_order'] = invals



In [344]:
innums = ['d','d','d','d','d','d']
keep = [item for item in itertools.permutations(innums,6)]
keep = list(set(keep))
keep.sort(reverse=False)
no_cues_rsvps = [item for item in keep if item[0] != 1 and item[-1] !=1]

In [345]:
no_cues_rsvps

[('d', 'd', 'd', 'd', 'd', 'd')]

In [346]:
invals = pd.Series(no_cues_rsvps*12).values

In [347]:
random.shuffle(invals)

In [348]:
df[(df.target_presence == ('absent', 'absent')) ].shape

(12, 13)

In [349]:
df.loc[df.target_presence == ('absent', 'absent'), 'rsvp_order'] = invals

In [350]:
df[['first','second','third','fourth','fifth','sixth']] = df['rsvp_order'].apply(pd.Series)

In [351]:
df.head()

,repeat_num,target_presence1,target_presence2,target_category1,target_category2,first_target,second_target,scene,object,target_presence,target_category,target_identity,rsvp_order,first,second,third,fourth,fifth,sixth
0,1,absent,absent,False,False,False,False,woods,flower,"(absent, absent)","(False, False)","(False, False)","(d, d, d, d, d, d)",d,d,d,d,d,d
1,1,absent,absent,False,False,False,False,woods,shoe,"(absent, absent)","(False, False)","(False, False)","(d, d, d, d, d, d)",d,d,d,d,d,d
2,1,absent,absent,False,False,False,False,bathroom,shoe,"(absent, absent)","(False, False)","(False, False)","(d, d, d, d, d, d)",d,d,d,d,d,d
3,1,absent,absent,False,False,False,False,woods,chair,"(absent, absent)","(False, False)","(False, False)","(d, d, d, d, d, d)",d,d,d,d,d,d
4,1,absent,present,False,scene,False,scene,woods,car,"(absent, present)","(False, scene)","(False, woods)","(d, 1, d, d, d, d)",d,1,d,d,d,d


In [352]:
loc_dict = {'second':1,'third':2,'fourth':3,'fifth':4}
cat_dict = {'object':objects, 'scene':scenes}
for i,row in df.iterrows():
    distractors1 = random.sample(objects,3)   
    distractors2 = random.sample(scenes,3) 
    distractors = distractors1 + distractors2
    random.shuffle(distractors)
    
    #while not tmp:
    #    tmp = distractors[loc_dict[row['target_location']]] in cat_dict[row['target_category']]

    equal_scenes_objects = False
    while not equal_scenes_objects:
        while row['target_identity'][0] in distractors or row['target_identity'][1] in distractors:            

                distractors1 = random.sample(objects,3)   
                distractors2 = random.sample(scenes,3) 
                distractors = distractors1 + distractors2
                random.shuffle(distractors)
                        
        try:
            distractors[loc_dict[row['target_location']]] = row['target_identity']
            equal_scenes_objects = len(set.intersection(set(distractors), set(scenes))) == len(set.intersection(set(distractors), set(objects)))
        
        except KeyError:
            equal_scenes_objects = len(set.intersection(set(distractors), set(scenes))) == len(set.intersection(set(distractors), set(objects)))



    if not len(set.intersection(set(distractors), set(scenes))) == len(set.intersection(set(distractors), set(objects))):
        print distractors
    #print len(set.intersection(set(distractors), set(scenes))), len(set.intersection(set(distractors), set(objects)))
    for j,item in enumerate(['first','second','third','fourth','fifth','sixth']):
        if df.loc[i, item] == 'd':
            df.loc[i, item] = distractors[j]
            
            
        

In [353]:
# df.loc[df['first'] == 1, 'first'] = df.target_identity
# df.loc[df['second'] == 1, 'second'] = df.target_identity
# df.loc[df['third'] == 1, 'third'] = df.target_identity
# df.loc[df['fourth'] == 1, 'fourth'] = df.target_identity
# df.loc[df['fifth'] == 1, 'fifth'] = df.target_identity
# df.loc[df['sixth'] == 1, 'sixth'] = df.target_identity

In [354]:
for i,row in df.iterrows():
    x = 0
    num_targets = len([(j,item) for j,item in enumerate(row[['first','second','third','fourth','fifth','sixth']].values) if item == 1])
    for j,item in enumerate(['first','second','third','fourth','fifth','sixth']):
        if num_targets == 2:
            if df.loc[i, item] == 1 and x == 0:
                print row['target_identity'][0]
                df.loc[i, item] = row['target_identity'][0]
                x +=1
            elif df.loc[i, item] == 1 and x == 1:
                print row['target_identity'][1]
                df.loc[i, item] = row['target_identity'][1]
                x +=1
            else:
                pass
        elif num_targets == 1:
            if df.loc[i, item] == 1:
                target_id = [y for y in row['target_identity'] if y != False][0]
                print target_id
                df.loc[i, item] = target_id
            else:
                pass
            
        
                
#     num_targets = len([(j,item) for j,item in enumerate(row[['first','second','third','fourth','fifth','sixth']].values) if item == 1])
#     if num_targets == 1:
#         print row['target_identity']
#     elif num_targets == 2:
#         print row['target_identity'] 



woods
chair
coast
flower
car
bathroom
chair
desert
flower
desert
coast
car
shoe
desert
bathroom
flower
coast
chair
woods
flower
shoe
woods
flower
desert
flower
coast
bathroom
flower
car
desert
bathroom
chair
desert
flower
coast
car
shoe
desert
shoe
coast
shoe
coast
bathroom
chair
car
bathroom
bathroom
flower


In [355]:
df['cue_type'] = 'doublecue'

In [356]:
df['first_target_id'] = [item[0] for item in df['target_identity'].values]
df['second_target_id'] = [item[1] for item in df['target_identity'].values]

In [357]:
df_new = df[['first_target_id','first','second','third','fourth','fifth','sixth']]

matches = df_new[df_new.columns.difference(['first_target_id'])].eq(df_new['first_target_id'], axis = 0)
df_new["first_target_location"] = matches.apply(lambda x: ", ".join(x.index[np.where(x.tolist())]), axis = 1)

df_new["first_exists"] = df_new.drop("first_target_id", 1).eq(df_new["first_target_id"], 0).any(1)

df["first_exists"] = df_new["first_exists"]
df["first_target_location"] = df_new["first_target_location"]

df.loc[df["first_target_location"] == "", "first_target_location"] = False

/Users/Amyunimus/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Amyunimus/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [358]:
df_new = df[['second_target_id','first','second','third','fourth','fifth','sixth']]

matches = df_new[df_new.columns.difference(['second_target_id'])].eq(df_new['second_target_id'], axis = 0)
df_new["second_target_location"] = matches.apply(lambda x: ", ".join(x.index[np.where(x.tolist())]), axis = 1)

df_new["second_exists"] = df_new.drop("second_target_id", 1).eq(df_new["second_target_id"], 0).any(1)

df["second_exists"] = df_new["second_exists"]
df["second_target_location"] = df_new["second_target_location"]

df.loc[df["second_target_location"] == "", "second_target_location"] = False

/Users/Amyunimus/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Amyunimus/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [359]:
df.head()

,repeat_num,target_presence1,target_presence2,target_category1,target_category2,first_target,second_target,scene,object,target_presence,...,fourth,fifth,sixth,cue_type,first_target_id,second_target_id,first_exists,first_target_location,second_exists,second_target_location
0,1,absent,absent,False,False,False,False,woods,flower,"(absent, absent)",...,coast,flower,shoe,doublecue,False,False,False,False,False,False
1,1,absent,absent,False,False,False,False,woods,shoe,"(absent, absent)",...,woods,shoe,desert,doublecue,False,False,False,False,False,False
2,1,absent,absent,False,False,False,False,bathroom,shoe,"(absent, absent)",...,car,desert,coast,doublecue,False,False,False,False,False,False
3,1,absent,absent,False,False,False,False,woods,chair,"(absent, absent)",...,woods,shoe,car,doublecue,False,False,False,False,False,False
4,1,absent,present,False,scene,False,scene,woods,car,"(absent, present)",...,bathroom,coast,car,doublecue,False,woods,False,False,True,second


In [360]:
df["exists"] = zip(df["first_exists"],df["second_exists"])
df["target_location"] = zip(df["first_target_location"],df["second_target_location"])

In [361]:
df['rsvp_order'] = df['first'].astype(str) +","+ df['second'].astype(str) +","+ \
df['third'].astype(str)+","+ df['fourth'].astype(str)+","+ df['fifth'].astype(str) +","+ df['sixth'].astype(str)

In [362]:
df_to_add = df[['repeat_num','target_presence','cue_type','target_identity','first','second','third','fourth','fifth','sixth',\
    'target_location','target_category','exists','rsvp_order']]#'correct_response','stimulus_path','stimulus_ext',
#    'stimulus_positions','choices','key_mappings']]

In [363]:
df_to_add = df_to_add[df_to_add['target_presence'] ==('present','present')] #only include items where one or both of the uces was there#only include items where one or both of the uces was there

In [364]:
final_df.head()

,repeat_num,target_presence,cue_type,target_identity,first,second,third,fourth,fifth,sixth,target_location,target_category,exists,rsvp_order
0,1,absent,nocue,False,bathroom,woods,coast,shoe,car,flower,False,False,False,"bathroom,woods,coast,shoe,car,flower"
1,1,present,postcue,desert,bathroom,woods,desert,shoe,car,chair,third,scene,True,"bathroom,woods,desert,shoe,car,chair"
2,1,present,precue,desert,bathroom,woods,desert,shoe,car,chair,third,scene,True,"bathroom,woods,desert,shoe,car,chair"
3,1,present,postcue,flower,desert,flower,coast,bathroom,car,chair,second,object,True,"desert,flower,coast,bathroom,car,chair"
4,1,present,precue,desert,woods,desert,coast,shoe,chair,flower,second,scene,True,"woods,desert,coast,shoe,chair,flower"


In [365]:
final_df_wcatch = pd.concat([final_df,df_to_add])

In [366]:
#randomizes trials by repeat (3 here), so that every trial is scene once before repeating
final_df_wcatch = final_df_wcatch.groupby('repeat_num').apply(lambda x: x.sample(frac=1)).reset_index(drop=True)




In [367]:
final_df_wcatch['sixth'].value_counts()

desert      84
chair       73
shoe        67
flower      61
car         59
coast       57
woods       52
bathroom    39
Name: sixth, dtype: int64

In [368]:
final_df_wcatch['first_num'] = final_df_wcatch.groupby("first").cumcount()+1
final_df_wcatch['second_num'] = final_df_wcatch.groupby("second").cumcount()+1
final_df_wcatch['third_num'] = final_df_wcatch.groupby("third").cumcount()+1
final_df_wcatch['fourth_num'] = final_df_wcatch.groupby("fourth").cumcount()+1
final_df_wcatch['fifth_num'] = final_df_wcatch.groupby("fifth").cumcount()+1
final_df_wcatch['sixth_num'] = final_df_wcatch.groupby("sixth").cumcount()+1

In [369]:
final_df_wcatch['first_num'] = final_df_wcatch['first_num'].sample(frac=1).reset_index(drop=True)
final_df_wcatch['second_num'] = final_df_wcatch['second_num'].sample(frac=1).reset_index(drop=True)
final_df_wcatch['third_num'] = final_df_wcatch['third_num'].sample(frac=1).reset_index(drop=True)
final_df_wcatch['fourth_num'] = final_df_wcatch['fourth_num'].sample(frac=1).reset_index(drop=True)
final_df_wcatch['fifth_num'] = final_df_wcatch['fifth_num'].sample(frac=1).reset_index(drop=True)
final_df_wcatch['sixth_num'] = final_df_wcatch['sixth_num'].sample(frac=1).reset_index(drop=True)

In [370]:
# for each target prresent condition, take a random sample of rsvp orders
# The orders should be used in both post-cue and pre-cue

# (not in no cue since there is no target present orders)

# target absent ordesr should be selected and used in all three conditions. 

In [371]:
final_df_wcatch

,repeat_num,target_presence,cue_type,target_identity,first,second,third,fourth,fifth,sixth,target_location,target_category,exists,rsvp_order,first_num,second_num,third_num,fourth_num,fifth_num,sixth_num
0,1,absent,precue,shoe,bathroom,desert,chair,woods,car,flower,False,object,False,"bathroom,desert,chair,woods,car,flower",48,29,43,55,24,50
1,1,absent,precue,bathroom,desert,coast,flower,shoe,woods,car,False,scene,False,"desert,coast,flower,shoe,woods,car",28,28,26,32,11,32
2,1,absent,postcue,bathroom,woods,flower,chair,car,desert,coast,False,scene,False,"woods,flower,chair,car,desert,coast",21,44,54,23,30,48
3,1,absent,precue,flower,chair,bathroom,desert,car,coast,shoe,False,object,False,"chair,bathroom,desert,car,coast,shoe",23,62,20,37,42,35
4,1,absent,nocue,False,woods,coast,chair,shoe,car,desert,False,False,False,"woods,coast,chair,shoe,car,desert",52,10,1,17,35,43
5,1,present,postcue,woods,flower,car,woods,chair,coast,desert,third,scene,True,"flower,car,woods,chair,coast,desert",2,66,56,63,41,26
6,1,absent,postcue,desert,car,shoe,woods,bathroom,coast,flower,False,scene,False,"car,shoe,woods,bathroom,coast,flower",32,58,35,4,40,49
7,1,absent,precue,flower,bathroom,car,chair,woods,shoe,coast,False,object,False,"bathroom,car,chair,woods,shoe,coast",57,17,29,11,13,6
8,1,absent,postcue,bathroom,desert,coast,flower,shoe,woods,car,False,scene,False,"desert,coast,flower,shoe,woods,car",12,60,6,38,1,60
9,1,absent,postcue,chair,woods,shoe,desert,flower,car,coast,False,object,False,"woods,shoe,desert,flower,car,coast",19,39,59,41,7,61


In [372]:
pd.crosstab([final_df_wcatch.target_category], final_df_wcatch.cue_type, margins=True)

cue_type,doublecue,nocue,postcue,precue,All
target_category,,,,,
False,0,96,0,0,96
object,0,0,96,96,192
scene,0,0,96,96,192
"(object, scene)",6,0,0,0,6
"(scene, object)",6,0,0,0,6
All,12,96,192,192,492


In [373]:
sec_per_trial = (.900+.300+ (6*.10) + .300 +.900 + 2.5 + .5)
print sec_per_trial
total_trials = final_df_wcatch.shape[0]
total_mins = total_trials*sec_per_trial/60.
print total_trials
print total_mins

6.0
492
49.2


In [374]:
492/8.

61.5

In [375]:
sec_per_trial*62/60

6.2

In [376]:
df.head()

,repeat_num,target_presence1,target_presence2,target_category1,target_category2,first_target,second_target,scene,object,target_presence,...,sixth,cue_type,first_target_id,second_target_id,first_exists,first_target_location,second_exists,second_target_location,exists,target_location
0,1,absent,absent,False,False,False,False,woods,flower,"(absent, absent)",...,shoe,doublecue,False,False,False,False,False,False,"(False, False)","(False, False)"
1,1,absent,absent,False,False,False,False,woods,shoe,"(absent, absent)",...,desert,doublecue,False,False,False,False,False,False,"(False, False)","(False, False)"
2,1,absent,absent,False,False,False,False,bathroom,shoe,"(absent, absent)",...,coast,doublecue,False,False,False,False,False,False,"(False, False)","(False, False)"
3,1,absent,absent,False,False,False,False,woods,chair,"(absent, absent)",...,car,doublecue,False,False,False,False,False,False,"(False, False)","(False, False)"
4,1,absent,present,False,scene,False,scene,woods,car,"(absent, present)",...,car,doublecue,False,woods,False,False,True,second,"(False, True)","(False, second)"


In [377]:
df= final_df_wcatch

In [378]:
df.shape[0]/3/2/2

41

In [379]:
df.head()

,repeat_num,target_presence,cue_type,target_identity,first,second,third,fourth,fifth,sixth,target_location,target_category,exists,rsvp_order,first_num,second_num,third_num,fourth_num,fifth_num,sixth_num
0,1,absent,precue,shoe,bathroom,desert,chair,woods,car,flower,False,object,False,"bathroom,desert,chair,woods,car,flower",48,29,43,55,24,50
1,1,absent,precue,bathroom,desert,coast,flower,shoe,woods,car,False,scene,False,"desert,coast,flower,shoe,woods,car",28,28,26,32,11,32
2,1,absent,postcue,bathroom,woods,flower,chair,car,desert,coast,False,scene,False,"woods,flower,chair,car,desert,coast",21,44,54,23,30,48
3,1,absent,precue,flower,chair,bathroom,desert,car,coast,shoe,False,object,False,"chair,bathroom,desert,car,coast,shoe",23,62,20,37,42,35
4,1,absent,nocue,False,woods,coast,chair,shoe,car,desert,False,False,False,"woods,coast,chair,shoe,car,desert",52,10,1,17,35,43


In [380]:
## Experiment Constants ##
stimulus_positions = [(0,0)] #this may change per stimulus, it can be excluded here and drawn from the trial list
stimulus_path = 'img/'
stimulus_ext = 'png'
choices = ['a','b']
key_mappings = {'a':'absent','b':'present'}

## Trial Structure: 1) part order 2) part type, 3) timing of part, 4) fixation, 5) when to start responsetimer
trial_parts = [(1,  'cue',                0.9,   False,   False),
               (2,  'blank',              0.5,  'black',  False),
               (3,  'single-stim',        0.1, 'white',  False),
               (4,  'single-stim',        0.1, 'white',  False),
               (5,  'single-stim',        0.1, 'white',  False),
               (6,  'single-stim',        0.1, 'white',  False),
               (7,  'single-stim',        0.1, 'white',  False),
               (8,  'single-stim',        0.1, 'white',  False),
               (9,  'blank',              0.5,  'black',  False),
               (10, 'cue',                0.9,   False,   False),
               (11, 'response_separate',  2.5,   False,   True),
               (12, 'blank',              0.5,   'black',   False)
                ]

In [381]:
trial_part_keys = ['trial_part_order','trial_part_type','timing','fixation','start_response_timer']
trial_structure_data = []
for item in trial_parts:
    trial_structure_data.append(dict(zip(trial_part_keys,item)))

In [382]:
df['correct_response'] = df.target_presence
df['stimulus_path'] = stimulus_path
df['stimulus_ext'] = stimulus_ext
df['stimulus_positions'] = [stimulus_positions] * len(df)
df['choices'] = [choices] * len(df)
df['key_mappings'] = [key_mappings] * len(df)

In [383]:
df_dict = df.T.to_dict().values()    

In [384]:
final_df_wcatch[final_df_wcatch.cue_type == 'doublecue']

,repeat_num,target_presence,cue_type,target_identity,first,second,third,fourth,fifth,sixth,...,third_num,fourth_num,fifth_num,sixth_num,correct_response,stimulus_path,stimulus_ext,stimulus_positions,choices,key_mappings
16,1,"(present, present)",doublecue,"(shoe, desert)",flower,shoe,desert,chair,coast,car,...,14,46,52,7,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"
33,1,"(present, present)",doublecue,"(flower, desert)",shoe,flower,bathroom,desert,car,chair,...,24,16,5,76,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"
34,1,"(present, present)",doublecue,"(bathroom, flower)",coast,bathroom,woods,chair,flower,car,...,58,8,52,25,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"
133,1,"(present, present)",doublecue,"(coast, car)",desert,flower,coast,car,bathroom,chair,...,12,52,35,16,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"
232,2,"(present, present)",doublecue,"(bathroom, chair)",shoe,flower,car,bathroom,chair,desert,...,12,64,46,58,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"
254,2,"(present, present)",doublecue,"(flower, coast)",bathroom,woods,flower,desert,coast,car,...,56,74,43,35,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"
292,2,"(present, present)",doublecue,"(bathroom, flower)",coast,car,bathroom,woods,flower,desert,...,12,68,29,45,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"
293,2,"(present, present)",doublecue,"(car, desert)",shoe,bathroom,coast,car,desert,woods,...,16,37,13,3,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"
371,3,"(present, present)",doublecue,"(car, bathroom)",shoe,woods,car,bathroom,coast,desert,...,31,55,25,37,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"
466,3,"(present, present)",doublecue,"(shoe, coast)",bathroom,shoe,chair,desert,coast,woods,...,13,19,9,21,"(present, present)",img/,png,"[(0, 0)]","[a, b]","{u'a': u'absent', u'b': u'present'}"


In [385]:
# List of variables you want to keep from original stimulus generation file (will probably be most of them)
keep_vars = ['repeat_num','target_presence','correct_response','cue_type','target_identity',
             'target_category','target_location', 'choices', 'key_mappings']

scene_dict = {'woods':1,'bathroom':2,'desert':3,'coast':4}
object_dict = {'flower':1,'car':2,'shoe':3,'chair':4}

mapping_dict = {1:'precue',2:'blank',3:'first', 4:'second', 5:'third',6:'fourth',
                7:'fifth',8:'sixth',9:'blank',10:'postcue',11:'response_separate',12:'blank'}

trial_structure_data_list = []
all_trials = []
for i,trial in enumerate(df_dict):
    
    keep = {}
    for x in keep_vars:
        keep[x] = trial[x]   
    all_trials.append(keep)
    
    
    for j, part in enumerate(trial_structure_data):
        
        #Assign General Variables
        part['stimulus_positions'] = trial['stimulus_positions']
        part['stimulus_path'] = trial['stimulus_path']
        part['stimulus_ext'] = trial['stimulus_ext']
        
        #Assign Cue labels (at pre or postcue parts)
        if trial['cue_type'] == 'precue':
            if mapping_dict[j+1] == 'precue':
                part['stimulus'] = [trial['target_identity']]
                part['meg_code'] = 81
            elif mapping_dict[j+1] == 'postcue':
                part['stimulus'] = [False]
                part['meg_code'] = 180
        
        elif trial['cue_type'] == 'postcue':
            if mapping_dict[j+1] == 'precue':
                part['stimulus'] = [False]
                part['meg_code'] = 80
            elif mapping_dict[j+1] == 'postcue':
                part['stimulus'] = [trial['target_identity']]
                part['meg_code'] = 181
        
        elif trial['cue_type'] == 'nocue':
            if mapping_dict[j+1] == 'precue':
                part['stimulus'] = [False]
                part['meg_code'] = 80
            elif mapping_dict[j+1] == 'postcue':
                part['stimulus'] = [False]
                part['meg_code'] = 180
        
        elif trial['cue_type'] == 'doublecue':
            if mapping_dict[j+1] == 'precue':
                part['stimulus'] = [trial['target_identity'][0]]
                part['meg_code'] = 81
            elif mapping_dict[j+1] == 'postcue':
                part['stimulus'] = [trial['target_identity'][1]]
                part['meg_code'] = 181
                
        #Assign stimuli
        for stim,stim_num in zip(['first','second','third','fourth','fifth','sixth'],
                                 ['first_num','second_num','third_num','fourth_num','fifth_num','sixth_num']):
            if mapping_dict[j+1] == stim:
                part['stimulus'] = [trial[stim] +"_" + str(trial[stim_num])]
                if trial[stim] in objects:
                    cat_id = '1'
                    stim_id = str(object_dict[trial[stim]])
                elif trial[stim] in scenes:
                    cat_id = '2'
                    stim_id = str(scene_dict[trial[stim]])
                if trial[stim] == trial['target_identity']:
                    targ_id = '9'
                else:
                    targ_id = ""
                
                meg_code = int(cat_id+stim_id+targ_id)
                part['meg_code'] = meg_code

                                    
        #Assign blanks
        if mapping_dict[j+1] in ['blank', 'response_separate']:
            part['stimulus'] = [False]
            if mapping_dict[j+1] == 'response_separate':
                part['meg_code'] = 77
            else:
                part['meg_code'] = False
            
        
                    
    trial_structure_data_list.append(deepcopy(trial_structure_data))
    
    
    


In [386]:
for i,item in enumerate(zip(all_trials, trial_structure_data_list)):
    all_trials[i]['trial_structure_data'] = item[1]
    all_trials[i]['trial_num'] = i+1
    

In [387]:
len(all_trials)

492

In [388]:
all_trials[1]

{'choices': ['a', 'b'],
 'correct_response': 'absent',
 'cue_type': 'precue',
 'key_mappings': {'a': 'absent', 'b': 'present'},
 'repeat_num': 1,
 'target_category': 'scene',
 'target_identity': 'bathroom',
 'target_location': False,
 'target_presence': 'absent',
 'trial_num': 2,
 'trial_structure_data': [{'fixation': False,
   'meg_code': 81,
   'start_response_timer': False,
   'stimulus': ['bathroom'],
   'stimulus_ext': 'png',
   'stimulus_path': 'img/',
   'stimulus_positions': [(0, 0)],
   'timing': 0.9,
   'trial_part_order': 1,
   'trial_part_type': 'cue'},
  {'fixation': 'black',
   'meg_code': False,
   'start_response_timer': False,
   'stimulus': [False],
   'stimulus_ext': 'png',
   'stimulus_path': 'img/',
   'stimulus_positions': [(0, 0)],
   'timing': 0.5,
   'trial_part_order': 2,
   'trial_part_type': 'blank'},
  {'fixation': 'white',
   'meg_code': 23,
   'start_response_timer': False,
   'stimulus': ['desert_28'],
   'stimulus_ext': 'png',
   'stimulus_path': 'img/'

In [389]:
import pickle

with open(subject+'_all_trials.pkl', 'wb') as handle:
    pickle.dump(all_trials, handle, protocol=pickle.HIGHEST_PROTOCOL)




In [390]:
for item in all_trials:
    if item['cue_type'] == 'precue':
        print item['stimulus']
        print ""
        print ""

KeyError: 'stimulus'

In [ ]:
with open('s007_all_trials.pkl', 'rb') as handle:
    tmp = pickle.load(handle)
    


In [ ]:
tmp[1]